<a href="https://colab.research.google.com/github/auramolina/Analitica-en-recursos-humanos/blob/main/Modelado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='0C2054'><b>Librerías

In [ ]:
#### Cargar paquetes
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model, tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, get_scorer_names, accuracy_score,confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler ## escalar variables

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#metrics
from sklearn.metrics import accuracy_score, classification_report

import joblib  ### para guardar modelos
import openpyxl
import sys
import os

In [ ]:
## Conectar el drive
from google.colab import drive
drive.mount('/content/drive')

path='/content/drive/MyDrive/AA/Analitica-en-recursos-humanos'
os.chdir(path) ### setting a Working directory on drive path
sys.path.append(path+'/')  ###setting path to read user defined functions

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import funciones_jc as fn ### importar modulo de  funciones
from funciones_jc import * ### importar toddas llas funciones del archivo

## con esto ya ses pueden utilizar todas las funciones propias

In [ ]:
d2015 = pd.read_csv('https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/Datos/dummies2015')
d2015.sort_values(by=['EmployeeID'],ascending=1)

,Age,DistanceFromHome,Education,EmployeeID,Gender,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,51,6,2,1,0,1,131160,1.0,11,0,...,False,False,False,False,False,False,False,False,True,False
1,31,10,1,2,0,1,41890,0.0,23,1,...,False,False,False,False,True,False,False,False,False,True
2,32,17,4,3,1,4,193280,1.0,15,3,...,False,False,False,False,False,True,False,False,True,False
3,38,2,5,4,1,3,83210,3.0,11,3,...,False,False,False,False,False,False,False,False,True,False
4,32,10,1,5,1,1,23420,4.0,12,2,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,29,7,1,8792,0,1,21800,1.0,21,0,...,False,False,False,False,True,False,False,False,False,True
4386,33,11,4,8797,1,1,71400,5.0,21,0,...,False,False,False,False,True,False,False,False,True,False
4388,33,1,3,8799,1,2,51470,7.0,11,0,...,False,True,False,False,False,False,False,False,True,False
4391,32,23,1,8802,1,3,24680,0.0,11,0,...,False,False,False,False,False,False,False,False,False,True


#<font color='0C2054'><b> Separación variables

In [ ]:
### Variable respuesta
y = d2015['Attrition_Yes']

## Variables explicativas
X1 = d2015.loc[:,~d2015.columns.isin(['Attrition_Yes'])]

## <font color='0C2054'>Estandarización

In [ ]:
## Estandarización y escalado de los datos
scaler = StandardScaler()
scaler.fit(X1)
X2 = scaler.transform(X1)
X = pd.DataFrame(X2,columns=X1.columns)

# <font color='0C2054'><b>Modelos Iniciales</font></b>

* Regresión logística: se selecciona LR por su fácil implementación, no requiere ajuste de hiperparametros y, es bueno en problemas de clasificación binaria.
* Bosques aleatorios: se selecciona RFC por su rendimiento, que generalmente es bueno, dando resultados satisfactorios, también porque es bueno en modelos que tienen muchas variables categóricas.

In [ ]:
m_lr = LogisticRegression()
m_rf = RandomForestClassifier()
m_gbt = GradientBoostingClassifier()

In [ ]:
modelos = list([m_lr, m_rf,m_gbt])

## <font color='0C2054'>Selección de variables

In [ ]:
var_names = fn.sel_variables(modelos, X, y, threshold = "2*mean")
var_names.shape

(5,)

In [ ]:
X2 = X[var_names] ### matriz con variables seleccionadas
X2.info()
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   resignationReason_Others    4410 non-null   float64
 1   resignationReason_Salary    4410 non-null   float64
 2   resignationReason_Stress    4410 non-null   float64
 3   retirementType_Fired        4410 non-null   float64
 4   retirementType_Resignation  4410 non-null   float64
dtypes: float64(5)
memory usage: 172.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 49 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                4410 non-null   float64
 1   DistanceFromHome                   4410 non-null   float64
 2   Education                          4410 non-null   float64
 3   EmployeeID           

# <font color='0C2054'><b>Modelado

## <font color='0C2054'>Calculo F1-score

In [ ]:
# Calcular F1-score con 30 iteraciones para todos los modelos con todas las variables
f1score_df = funciones.medir_modelos(modelos, "f1", X, y, 30)
# Calcular F1-score con 30 iteraciones para todos los modelos con variables seleccionadas
f1_score_varsel = funciones.medir_modelos(modelos, "f1", X2, y, 30)
# visualizar los resultados de f1- score para los modelos con todas las variabels y con las variables seleccionadas
f1=pd.concat([f1score_df,f1_score_varsel],axis=1)
f1.columns=['rl', 'rf', 'gb', 'rl_Sel', 'rf_sel', 'gb_Sel']
f1

In [ ]:
# promedio de la metrica F1-score para todos los modelos con todas las variables y con las variables seleccionadas
f1.mean()

## <font color='0C2054'>Grilla de hiperparametros para el modelo Ramdon Forest

In [ ]:
param_grid_rf = {
    'n_estimators': [5, 50, 100, 200, 500],  # Número de árboles en el bosque
    'max_features': [5, 20],  # Número de características a considerar en cada división
    'max_depth': [10, 20, 30],  # Profundidad máxima del árbol
    'min_samples_split': [5, 20, 100],  # Número mínimo de muestras requeridas para dividir un nodo interno
    'min_samples_leaf': [1, 2, 4],  # Número mínimo de muestras requeridas para estar en un nodo hoja
    'bootstrap': [True, False]  # Método de muestreo para construir árboles
}

## <font color='0C2054'>Grilla de hiperparámetros para Gradient Boosting

In [ ]:
param_grid_gb = {
    'n_estimators': [100, 200, 500],  # Número de árboles en el ensemble
    'learning_rate': [0.01, 0.1, 0.2, 0.3],  # Tasa de aprendizaje
    'max_depth': [3, 5, 8],  # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras necesarias para dividir un nodo
    'min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras necesarias en un nodo hoja
    'subsample': [0.8, 0.9, 1.0],  # Fracción de muestras utilizadas para entrenar cada árbol
    'max_features': ['auto', 'sqrt', 'log2']  # Número de características a considerar al hacer cada división
}

## <font color='0C2054'>Técnica de búsqueda de hiperparámetros para el modelo Ramdon Forest

In [ ]:
tun_rf = RandomizedSearchCV(m_rf,param_distributions=param_grid_rf,n_iter=6,scoring="f1")
tun_rf.fit(X2,y)

In [ ]:
# Imprimir los parametros de la grilla con el mejor puntaje
pd.set_option('display.max_colwidth', 100)
resultados=tun_rf.cv_results_
tun_rf.best_params_
pd_resultados=pd.DataFrame(resultados)
pd_resultados[["params","mean_test_score"]].sort_values(by="mean_test_score", ascending=False)

## <font color='0C2054'>Técnica de búsqueda de hiperparámetros para el GradientBoostingClassifier

In [ ]:
# Crear la instancia de GridSearchCV para búsqueda de hiperparámetros
tun_gb = GridSearchCV(estimator=m_gbt, param_grid=param_grid_gb, scoring='f1', cv=5)
# Realizar la búsqueda de hiperparámetros en los datos
tun_gb.fit(X2, y)

In [ ]:
# Imprimir los parametros de la grilla con el mejor puntaje
pd.set_option('display.max_colwidth', 100)
resultados=tun_gb.cv_results_
tun_gb.best_params_
pd_resultados=pd.DataFrame(resultados)
pd_resultados[["params","mean_test_score"]].sort_values(by="mean_test_score", ascending=False)

In [ ]:
# Mejor grilla del modelo GradientBoostingClassifier
dt_final = tun_gb.best_estimator_

# Guardar el modelo con hyperparameter tunning
rf_final = tun_rf.best_estimator_

# <font color='0C2054'><b>Analisis del los modelos

In [ ]:
#####Evaluar métrica de entrenamiento y evaluación para mirar sobre ajuste ####
eval_gb=cross_validate(gb_final,X2,y,cv=30,scoring="f1",return_train_score=True)
eval_rf=cross_validate(rf_final,X2,y,cv=30,scoring="f1",return_train_score=True)

In [ ]:
#### convertir resultado de evaluacion entrenamiento y evaluacion en data frame para Ramdon Forest
train_rf=pd.DataFrame(eval_rf['train_score'])
test_rf=pd.DataFrame(eval_rf['test_score'])
train_test_rf=pd.concat([train_rf, test_rf],axis=1)
train_test_rf.columns=['train_score','test_score']
train_test_rf

In [ ]:
#### convertir resultado de evaluacion entrenamiento y evaluacion en data frame para Ramdon Forest
train_gb=pd.DataFrame(eval_gb['train_score'])
test_gb=pd.DataFrame(eval_gb['test_score'])
train_test_gb=pd.concat([train_gb, test_gb],axis=1)
train_test_gb.columns=['train_score','test_score']
train_test_gb